# IBIT Backtest: AMMA vs Trend + Mining + OU

This notebook curates IBIT and cleaned crypto mining-cost data, aligns dates, and compares:
- **AMMA** strategy (fit to `IBIT_Data.csv`)
- **Buy & Hold** on IBIT
- **Trend + Mining + OU** ensemble strategy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Backtest.amma import amma_from_ibit_csv
from Models.trend import trend_signal
from Models.mining import mining_signal
from Models.ou import ou_signal

plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
IBIT_PATH = '../Data/IBIT_Data.csv'
CLEANED_PATH = '../Data/cleaned_crypto_data.csv'

ibit = pd.read_csv(IBIT_PATH)
cleaned = pd.read_csv(CLEANED_PATH)

ibit['Date'] = pd.to_datetime(ibit['Date'], format='%m/%d/%y', errors='coerce')
ibit['IBIT_price'] = (
    ibit['Price']
    .astype(str)
    .str.replace(',', '', regex=False)
    .str.replace('$', '', regex=False)
    .astype(float)
)

cleaned['Date'] = pd.to_datetime(cleaned['Date'], errors='coerce')

curated = (
    ibit[['Date', 'IBIT_price']]
    .merge(cleaned[['Date', 'COST_TO_MINE']], on='Date', how='inner')
    .dropna(subset=['Date', 'IBIT_price', 'COST_TO_MINE'])
    .sort_values('Date')
    .reset_index(drop=True)
)

curated['ret'] = curated['IBIT_price'].pct_change().fillna(0.0)
curated.head(), curated[['Date','IBIT_price','COST_TO_MINE']].describe(include='all')

In [ ]:
# AMMA on IBIT
momentum_weights = {5: 0.25, 20: 0.35, 60: 0.40}
amma_df = amma_from_ibit_csv(
    ibit_csv_path=IBIT_PATH,
    momentum_weights=momentum_weights,
    threshold=0.0,
    long_enabled=True,
    short_enabled=False,
)

amma_curated = curated[['Date', 'ret']].merge(
    amma_df[['Date', 'amma_position']],
    on='Date',
    how='left'
).fillna({'amma_position': 0.0})

amma_curated['amma_ret'] = amma_curated['amma_position'] * amma_curated['ret']

In [ ]:
# Trend + Mining + OU overlays (AMMA as base), aligned by date
crypto = curated.copy()
crypto['BTC-USD_close'] = crypto['IBIT_price']
crypto['IBIT_close'] = crypto['IBIT_price']
crypto['amma'] = amma_curated['amma_position'].values.clip(0.0, 1.0)

crypto['trend_raw'] = trend_signal(
    crypto,
    price_column='BTC-USD_close',
    fast_window=20,
    slow_window=128,
    long_only=True,
)

crypto['mining_raw'] = mining_signal(
    crypto,
    z_window=180,
    entry_z=0.5,
    exit_z=0.0,
    use_log_edge=True,
)

crypto['ou_raw'] = ou_signal(
    crypto['BTC-USD_close'],
    window=90,
    entry_z=1.5,
    exit_z=0.3,
    long_short=False,
    detrend_window=180,
)

print('BTC close tail:')
print(crypto['BTC-USD_close'].tail())

print('
trend_raw stats:')
print(crypto['trend_raw'].describe())
print('trend_raw unique tail:', crypto['trend_raw'].tail().unique())

print('
ou_raw stats:')
print(crypto['ou_raw'].describe())

print('
mining_raw stats:')
print(crypto['mining_raw'].describe())

# Robust trend exposure from BTC close (no dependence on trend_raw scaling)
fast = crypto['BTC-USD_close'].ewm(span=20, adjust=False).mean()
slow = crypto['BTC-USD_close'].ewm(span=128, adjust=False).mean()

# 1 when fast > slow else 0
crypto['trend_expo'] = (fast > slow).astype(float)

# Soften edges (still no leverage): scale by trend magnitude
trend_strength = ((fast / slow) - 1.0).abs()
crypto['trend_expo'] = (0.5 + 20 * trend_strength).clip(0.0, 1.0) * crypto['trend_expo']

# Build risk from OU/Mining overlays
crypto['ou_risk'] = (1.0 - crypto['ou_raw']).clip(0.0, 1.0)
crypto['mining_risk'] = (1.0 - crypto['mining_raw']).clip(0.0, 1.0)
risk = (0.55 * crypto['ou_risk'] + 0.45 * crypto['mining_risk']).clip(0.0, 1.0)

# Base = AMMA. Trend + risk overlays only throttle the base exposure.
base = crypto['amma'].clip(0.0, 1.0)
trend_gate = (0.35 + 0.65 * crypto['trend_expo']).clip(0.0, 1.0)
risk_throttle = (1.0 - 0.60 * risk).clip(0.20, 1.0)

crypto['combo_expo'] = (base * trend_gate * risk_throttle).clip(0.0, 1.0)
crypto['combo_expo'] = crypto['combo_expo'].rolling(5, min_periods=1).mean()
crypto['combo_ret'] = crypto['combo_expo'].shift(1).fillna(0.0) * crypto['ret']

print('
Any NaNs in BTC close?', crypto['BTC-USD_close'].isna().sum())
print('Any NaNs in IBIT_close?', crypto['IBIT_close'].isna().sum())
print('Last 10 dates in crypto index:', crypto.index[-10:].tolist())

strategy = crypto[['Date', 'ret', 'trend_expo', 'ou_risk', 'mining_risk', 'combo_expo', 'combo_ret']].copy()
crypto[['combo_expo', 'amma', 'trend_expo', 'ou_risk', 'mining_risk']].tail()


In [ ]:
# Buy & Hold baseline
results = curated[['Date']].copy()
results['buy_hold_ret'] = curated['ret']
results['amma_ret'] = amma_curated['amma_ret'].values
results['combo_ret'] = strategy['combo_ret'].values

for c in ['buy_hold_ret', 'amma_ret', 'combo_ret']:
    results[f'{c}_equity'] = (1 + results[c]).cumprod()

results.tail()

In [ ]:
def perf_summary(returns, days=252):
    r = pd.Series(returns).dropna()
    if len(r) < 2:
        return {'CAGR': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan}

    equity = (1 + r).cumprod()
    years = len(r) / days
    cagr = equity.iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan
    vol = r.std(ddof=1) * np.sqrt(days)
    sharpe = (r.mean() * days) / vol if vol > 0 else np.nan
    dd = equity / equity.cummax() - 1

    return {'CAGR': cagr, 'Sharpe': sharpe, 'MaxDD': dd.min()}

summary = pd.DataFrame({
    'Buy & Hold': perf_summary(results['buy_hold_ret']),
    'AMMA': perf_summary(results['amma_ret']),
    'Trend+Mining+OU': perf_summary(results['combo_ret']),
}).T

summary

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(results['Date'], results['buy_hold_ret_equity'], label='Buy & Hold', linewidth=2)
plt.plot(results['Date'], results['amma_ret_equity'], label='AMMA', linewidth=2)
plt.plot(results['Date'], results['combo_ret_equity'], label='Combo (AMMA gated by Trend + Mining + OU)', linewidth=2)
plt.title('IBIT Equity Curve Comparison')
plt.xlabel('Date')
plt.ylabel('Equity (Start = 1.0)')
plt.legend()
plt.tight_layout()
plt.show()